<a href="https://colab.research.google.com/github/dominikklepl/News-search-engine/blob/master/Code_explanation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News search engine
In this notebook we'll go through all code for building our own search engine specialised on news articles.
Each search engine consists of three main components:


* Crawler
* Indexer
* Query processor



## A. Crawler
We start with building crawler that goes to RSS feed, extracts title, description (if any), date published and link. Processes this information and saves in a meta-data "database" (actually a pandas dataframe and stores it as csv file.

### Import packages

In [1]:
!pip install feedparser

import feedparser
import pandas as pd

     |████████████████████████████████| 194kB 10.3MB/s 
  Created wheel for feedparser: filename=feedparser-5.2.1-cp36-none-any.whl size=44940 sha256=baa1516fef5a91ceef4a7192d36d52a9e3a34d58a04487cf2ab50014918f860e
  Stored in directory: /root/.cache/pip/wheels/8c/69/b7/f52763c41c5471df57703a0ef718a32a5e81ee35dcf6d4f97f
Successfully built feedparser


Read RSS feed from a URL. Also keep track of how many news are in the feed and how many of these have already been scraped with previous iteration of the crawler. This will be later important to automatically adjust the frequency with which the crawler will visit the RSS feed to scrape new information.
For testing we use just one URL: BBC World News.

In [2]:
URL = "http://feeds.bbci.co.uk/news/world/rss.xml"
feed = feedparser.parse(URL)

feed_len = len(feed.entries) #number of news in feed
old_news = 0  # count how many news in feed were already scraped

print("There are {} news in the RSS feed." .format(feed_len))

There are 28 news in the RSS feed.


Load the meta-data database stored as csv file. If this is the first time the crawler is let loose this will be just an empty file with prepared column names (ID, title, date and link).
I'll keep it commented out here and instead just create an empty dataframe a this point.

In [3]:
#meta_data = pd.read_csv(PATH + "database.csv", index_col = 'Unnamed: 0')
meta_data = pd.DataFrame(columns=['ID', 'title', 'summary', 'link', 'published'])
meta_data.head()

,ID,title,summary,link,published


Now we write a function that accepts one entry from the feed and parse its contents to list with title, date published and link and assigns it a unique ID (which also denotes when was the entry scraped and entered to our search engine).
We normalize the date so that it's in format day/month/year. We don't care about more precise time.

In [0]:
def process_entry(entry, ID):
  ID = ID
  title = entry.title
  summary = entry.summary
  link = entry.link
  published = str(entry.published_parsed.tm_mday) + '/' + \
              str(entry.published_parsed.tm_mon) + '/' + \
              str(entry.published_parsed.tm_year)
  return [ID, title, summary, link, published]

Test the function on one entry

In [5]:
test_entry = feed.entries[0]
process_entry(test_entry, 1)

[1,
 "Data leak reveals how China 'brainwashes' Uighurs in prison camps",
 "Leaked documents show new evidence of China's systematic brainwashing of Uighur and other detainees.",
 'https://www.bbc.co.uk/news/world-asia-china-50511063',
 '24/11/2019']

Now we iterate over all entries in the feed. Check if the entry is already in the meta-data database, if not the entry is processed, assigned an ID and appended to a list of entries that will be later added to the database.

In [0]:
data = [] #dataframe for saving the entries
n = len(meta_data)+1 #ID value based on the highest ID value in database
for i in range(len(feed.entries)):
  entry = feed.entries[i]
  
  #check that link isn't in the database yet
  if entry.link not in meta_data['link'].values:
    processed = process_entry(entry = entry, ID=n)
    data.append(processed)
    n += 1 #increase the ID value
  else: old_news += 1 #count already scraped entries

If there was at least one newly scraped entry, we add it to the database.

In [0]:
if len(data) > 0:
  #transform data to pandas DataFrame
  news_extracted = pd.DataFrame(data, columns=['ID', 'title', 'summary', 'link', 'published'])

  #add new news to the database
  meta_data = pd.concat([meta_data, news_extracted], axis = 0)

  #write database to a csv file
  #meta_data.to_csv(PATH + "database.csv")

Look at the database

In [8]:
meta_data.head(5)

,ID,title,summary,link,published
0,1,Data leak reveals how China 'brainwashes' Uigh...,Leaked documents show new evidence of China's ...,https://www.bbc.co.uk/news/world-asia-china-50...,24/11/2019
1,2,Hong Kong elections: Pro-democracy group 'make...,Opposition candidates have attracted support a...,https://www.bbc.co.uk/news/world-asia-china-50...,24/11/2019
2,3,DR Congo: Many dead as plane crashes into homes,At least 27 people are dead after a passenger ...,https://www.bbc.co.uk/news/world-africa-50536220,24/11/2019
3,4,Floods cause chaos in France and Italy,A bridge collapsed near the Italian city of Sa...,https://www.bbc.co.uk/news/world-europe-50536502,24/11/2019
4,5,"Queen Hind cargo ship carrying 14,000 sheep ov...",An operation to rescue the animals is under wa...,https://www.bbc.co.uk/news/world-europe-50538592,24/11/2019


Get percentage of already-scraped entries

In [9]:
print("{} % of entries were already scraped." .format((old_news/feed_len)*100))

0.0 % of entries were already scraped.


## B. Indexer
Second part of a search engine is an indexer. It's basically a smart storage of our news articles in which we can later easily retrieve relative articles given a search query.
It parses the title and description of the news articles scraped by the crawler to single words. All these words make up the vocabulary of our index. Next step is to put the ID of the article in the posting lists of the words that the article contains. For example article called "This happened today" will be stored in posting lists of terms "this", "happened" and "today".
Before creating the index we preprocess the text of the articles in order to get rid of useless information. We the text of accents and turn everything to lowercase. Next we perform lemmatization. This is slightly smarter version of stemming. Essentially, it's a word normalization, e.g. all nouns to singular, all verbs in present tense etc.

Let's do it.

#### Importing

In [10]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer 
import string

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


We start with an empty dictionary as our index. As we scrape more articles later we will instead of starting with an empty index just update the already created index.

The index is organized as:
```
{
  "word1": \[ID1, ID2, ...],
  "word2": \[ID5, ID8, ...],
  ...
}
```

We start with just a single article from our meta-data database.

In [0]:
entry = meta_data.loc[0,:].copy()

In [12]:
entry

ID                                                           1
title        Data leak reveals how China 'brainwashes' Uigh...
summary      Leaked documents show new evidence of China's ...
link         https://www.bbc.co.uk/news/world-asia-china-50...
published                                           24/11/2019
Name: 0, dtype: object

### Text preprocessing
Turn title to lowercase, remove accents

In [0]:
def process_string(text):
  text = text.lower() #to lowercase
  text = text.translate(str.maketrans('', '', string.punctuation)) #strip punctuation
  return text

In [14]:
process_string(entry.title)

'data leak reveals how china brainwashes uighurs in prison camps'

Now, lemmatize, i.e. word normalization.

This method requires some additional information about the words. We need to find the word category of each word, e.g. verb, noun etc.

In [0]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

Test the function

In [16]:
print("Apple: {}\n Run: {}\n Happy: {}" .format(get_wordnet_pos("apple"), get_wordnet_pos("run"), get_wordnet_pos("happy")))

Apple: n
 Run: v
 Happy: a


We also need to remove stopwords, i.e. words with low informational value.

In [0]:
stop = stopwords.words('english')

Now we'll iterate over all words in text, lemmatize and return the transformed string.

In [0]:
lem = WordNetLemmatizer()

def stop_lemmatize(doc):
    tokens = nltk.word_tokenize(doc)
    tmp = ""
    for w in tokens:
        if w not in stop:
            tmp += lem.lemmatize(w, get_wordnet_pos(w)) + " "
    return tmp

In [19]:
stop_lemmatize(doc = entry.title)

"Data leak reveals China 'brainwashes ' Uighurs prison camp "

In [0]:
def process_string(text):
  text = text.lower() #to lowercase
  text = text.translate(str.maketrans('', '', string.punctuation)) #strip punctuation
  text = stop_lemmatize(text)
  return text

In [21]:
%time process_string(entry.title)

CPU times: user 4.25 ms, sys: 0 ns, total: 4.25 ms
Wall time: 5.08 ms


'data leak reveals china brainwashes uighur prison camp '

Now we apply the process_string function to all titles and summaries in our database.

In [0]:
meta_processed = meta_data.copy()

In [0]:
def transform_df(df):
  df['title'] = df['title'].apply(process_string)
  df['summary'] = df['summary'].apply(process_string)

In [24]:
%time transform_df(meta_processed)

CPU times: user 88.9 ms, sys: 5.55 ms, total: 94.4 ms
Wall time: 100 ms


In [25]:
meta_processed.head(5)

,ID,title,summary,link,published
0,1,data leak reveals china brainwashes uighur pri...,leak document show new evidence china systemat...,https://www.bbc.co.uk/news/world-asia-china-50...,24/11/2019
1,2,hong kong election prodemocracy group make big...,opposition candidate attract support expense p...,https://www.bbc.co.uk/news/world-asia-china-50...,24/11/2019
2,3,dr congo many dead plane crash home,least 27 people dead passenger plane hit resid...,https://www.bbc.co.uk/news/world-africa-50536220,24/11/2019
3,4,flood cause chaos france italy,bridge collapse near italian city savona sever...,https://www.bbc.co.uk/news/world-europe-50536502,24/11/2019
4,5,queen hind cargo ship carry 14000 sheep overtu...,operation rescue animal way ship capsize roman...,https://www.bbc.co.uk/news/world-europe-50538592,24/11/2019


In practice, we won't be transforming the whole meta-data database since that would mean creating index from scratch after every crawler iteration. Instead we would use only subset of the database with only newly added articles.

Now we can iterate over all entries to create the index. We'll go step by step again before wrapping it all in one nice function.

Merge title and summary into one field and drop all columns except for ID as we don't need those anymore.

In [0]:
meta_processed['text'] = meta_processed['title'] + " " + meta_processed['summary']
drop_cols = ['title', 'summary', 'published', 'link']
meta_processed = meta_processed.drop(drop_cols, axis=1)

In [27]:
meta_processed.head(5)

,ID,text
0,1,data leak reveals china brainwashes uighur pri...
1,2,hong kong election prodemocracy group make big...
2,3,dr congo many dead plane crash home least 27 ...
3,4,flood cause chaos france italy bridge collaps...
4,5,queen hind cargo ship carry 14000 sheep overtu...


Add this part to a transform_df function.

In [0]:
def transform_df(df):
  df = df
  df['title'] = df['title'].apply(process_string)
  df['summary'] = df['summary'].apply(process_string)
  df['text'] = df['title'] + " " + df['summary']
  drop_cols = ['title', 'summary', 'published', 'link']
  df = df.drop(drop_cols, axis=1)
  return df

### Build index

Now we'll build index with just one entry.

In [29]:
entry = meta_processed.loc[0,:].copy()
print(entry)
index_test = {}

ID                                                      1
text    data leak reveals china brainwashes uighur pri...
Name: 0, dtype: object


Split the entry to single words and return list and save entry's ID as object.

In [0]:
words = entry.text.split()
ID = entry.ID

Each word in index' vocabulary is a dictionary key and has its own posting list with IDs. Let's construct one word vocabulary as example.

In [31]:
word = words[0]
sample = {word: [ID]}
print(sample)

{'data': [1]}


Now we iterate over all words and if they aren't in the vocabulary yet we add them. Also for each word we append the entry ID to the posting list.

In [0]:
for word in words:
  if word in index_test.keys():
    if ID not in index_test[word]:
      index_test[word].append(ID)
  else:
    index_test[word] = [ID]

In [33]:
print(index_test)

{'data': [1], 'leak': [1], 'reveals': [1], 'china': [1], 'brainwashes': [1], 'uighur': [1], 'prison': [1], 'camp': [1], 'document': [1], 'show': [1], 'new': [1], 'evidence': [1], 'systematic': [1], 'brainwashing': [1], 'detainee': [1]}


Now this process can be repeated for all entries in the database

In [0]:
def index_it(entry, index):
  words = entry.text.split()
  ID = entry.ID
  for word in words:
    if word in index.keys():
      if ID not in index[word]:
        index[word].append(ID)
    else:
      index[word] = [ID]
  return index

In [35]:
ind = index_it(entry=entry, index= {})
print(ind)

{'data': [1], 'leak': [1], 'reveals': [1], 'china': [1], 'brainwashes': [1], 'uighur': [1], 'prison': [1], 'camp': [1], 'document': [1], 'show': [1], 'new': [1], 'evidence': [1], 'systematic': [1], 'brainwashing': [1], 'detainee': [1]}


Again we can iterate over all entries in the database with scraped articles, process them append to index.

In [0]:
def index_all(df, index):
  for i in range(len(df)):
    entry = df.loc[i,:]
    index = index_it(entry = entry, index = index)
  return index

In [37]:
index = index_all(meta_processed, index = {})
len(index)

376

Finally we wrap everything in one nice function.

In [0]:
def build_index(df, index):
    to_add = transform_df(df)
    index = index_all(df = to_add, index = index)
    return index

In [0]:
idx = build_index(df = meta_data, index = {})

In [40]:
len(idx)

376

And for future use we save the index to json file.

In [0]:
import json

with open('index.json', 'w') as fp:
    json.dump(idx, fp, sort_keys=True, indent=4)

It can be of course opened again with following code.

In [0]:
with open('index.json', 'r') as f:
    data = json.load(f)

# Bb Ranked retrieval
  The user would probably prefer the more relevant pages to be displayed before those that are less relevant (hopefully they're at least a bit relevant).
For our search engine to support such option we need to store some information about the scraped documents that could be later used for this purpose.
We'll use averaged word2vec for this purpose. Word2Vec model is single hidden-layer neural network. The hidden layer is actually what is so useful about this model. Given a word the layer's activation gives a unique vector that word. For each document we can iterate over all words, extract their vectors and then by averaging obtain a document vector. 
  Compared to other methods averaged word2vec has multiple advantages. Unlike simpler methods such as bag-of-words, n-grams and tf-idf the size of the vectors is fixed. For example bag-of-words is also using vectors but the size of these vectors equals the number of unique words in the corpus. This means that the computational and storage requirements get larger as the corpus gets larger.
  Averaged word2vec is also able to represent the documents on more abstract level than simpler methods and should therefore provide better method of ranking.
  We're using word2vec rather than doc2vec because we can simply use pretrained word2vec model to compute the document vectors. Using doc2vec would mean training a neural network from scratch which requires computational power, time and rather large dataset. 

Import and download pretrained word2vec model

In [43]:
import gensim
import numpy as np
!wget https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz

--2019-11-24 19:49:56--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.169.229
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.169.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  35.8MB/s    in 45s     

2019-11-24 19:50:42 (35.0 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



Load word2vec model

In [44]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Try getting vectors for all words in the text and averaging to get single vector.

In [45]:
print(words)

['data', 'leak', 'reveals', 'china', 'brainwashes', 'uighur', 'prison', 'camp', 'leak', 'document', 'show', 'new', 'evidence', 'china', 'systematic', 'brainwashing', 'uighur', 'detainee']


In [0]:
def average_vectors(word2vec_model, doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc if word in word2vec_model.vocab]
    if len(doc) == 0:
      return np.zeros(300)
    else:
      return np.mean(word2vec_model[doc], axis=0)

In [47]:
%time test_vec = average_vectors(word2vec, words)

CPU times: user 718 µs, sys: 45 µs, total: 763 µs
Wall time: 554 µs


Now we can iterate over documents, compute their vectors and construct a document vectors database.

In [0]:
def prepare_ranking(df):
  corpus = df[['ID', 'text']].copy()
  doc_vecs = {}
  for i in range(len(corpus)):
    row = corpus.loc[i,:]
    text = row.text.split()
    doc_vecs[row.ID]=average_vectors(word2vec, text)
  doc_vecs = pd.DataFrame.from_dict(data=doc_vecs, orient="index")
  doc_vecs['ID'] = doc_vecs.index
  return doc_vecs

In [0]:
doc_vecs = prepare_ranking(df=meta_data)

## C. Query processor
The final part of a search engine is a query processor which actually performs the search task. Given a query by user the processor should return list of relevant documents.
There are multiple types of queries. We'll start with a simple "google-ish" query where assume the user looks for documents relevant to all words in the query. Therefore we transform the query to boolean by connecting all words with AND operator.

First, the processor preprocesses the query the same way as the indexer preprocessed the text. In other words, we normalize the query to match the format of text in the index. Next, the query is parsed to single words. We look into index if these words are part of the vocabulary. If a word is in index we retrieve its posting list. Finally, we look for intersection of all retrieved posting lists. The result is list of document IDs that the user asked for.
However, we need to return something more useful than just a list of IDs. Therefore,we retrieve the information stored about the documents in the meta-data database. Before printing the results we should also rank the documents. This ranking should be based on relevance to query.
Optionally, the user may ask for news only from limited time window, e.g. published today or last week. So we need to filter the retrieved documents if this happens.

-----
To implement:
 - Boolean query
 - phrase matching
 -----


### Normalize query

Let's define an example query.

In [0]:
test = "Trump Ukraine China"

Now we use the "process string" function from used by indexer to normalize the query

In [51]:
print("User query: {}." .format(test))
test_norm = process_string(test)
print("Normalized query: {}." .format(test_norm))

User query: Trump Ukraine China.
Normalized query: trump ukraine china .


Now we split the query into words.

In [0]:
test_split = test_norm.split()

And we wrap this in function

In [0]:
def process_query(query):
  norm = process_string(query)
  return norm.split()

### Retrieve from index

And we iterate over the words, looking if they're in the index vocabulary. If so then we retrieve the associated posting list.

In [0]:
retrieved = []
for word in test_split:
  if word in index.keys():
    retrieved.append(index[word])

Now we look for the intersection of all posting lists.

In [0]:
def lists_intersection(lists):
  intersect = list(set.intersection(*map(set, lists)))
  intersect.sort()
  return intersect

In [106]:
result = lists_intersection(retrieved)
print(result)

[]


Let's wrap this part in a function before proceeding to formatting the results. The additional if statement is for cases when there's nothing retrieved.

In [0]:
def search_googleish(query, index=idx):
  query_split = process_query(query)
  retrieved = []
  for word in query_split:
    if word in index.keys():
      retrieved.append(index[word])
  if len(retrieved)>0:
    result = lists_intersection(retrieved)
  else:
      result = [0]
  return result

In [88]:
result_IDs = search_googleish("Trump", index)
print(result_IDs)

[19]


-----

*TO DO:
If there's no document retrieved, try removing one term and looking for simplified query + tell user that such document doesn't include term X.*

-----

### Retrieve meta-data
Now we need to connect the retrieved IDs with some useful information stored in database that we first use to refine the results and then to print nice result to user.

In [59]:
#in real setting we'll read the database from file here
#meta = pd.read_csv("database.csv")

#this is our database
meta = meta_data.drop(['text'], axis=1).copy()
meta.head(5)

,ID,title,summary,link,published
0,1,data leak reveals china brainwashes uighur pri...,leak document show new evidence china systemat...,https://www.bbc.co.uk/news/world-asia-china-50...,24/11/2019
1,2,hong kong election prodemocracy group make big...,opposition candidate attract support expense p...,https://www.bbc.co.uk/news/world-asia-china-50...,24/11/2019
2,3,dr congo many dead plane crash home,least 27 people dead passenger plane hit resid...,https://www.bbc.co.uk/news/world-africa-50536220,24/11/2019
3,4,flood cause chaos france italy,bridge collapse near italian city savona sever...,https://www.bbc.co.uk/news/world-europe-50536502,24/11/2019
4,5,queen hind cargo ship carry 14000 sheep overtu...,operation rescue animal way ship capsize roman...,https://www.bbc.co.uk/news/world-europe-50538592,24/11/2019


Query from database to get only rows of retrieved IDs

In [0]:
def connect_id_df(retrieved_id, df):
    return df[df.ID.isin(retrieved_id)].reset_index(drop=True)

In [89]:
result_meta = connect_id_df(result_IDs, meta)
result_meta.head(5)

,ID,title,summary,link,published
0,19,week impeachment news three minute,congress heard explosive testimony president d...,https://www.bbc.co.uk/news/world-us-canada-505...,22/11/2019


### Ranked retrieval
Now we return back to the word2vec vectors we computed after indexing the documents.
We'l compute the vector for the query as well and then using a cosine similarity compare query to retrieved document relevance.

Compute vector for query

In [0]:
query_vec = average_vectors(word2vec, test_split)

Retrieve vectors of retrieve documents.

In [0]:
result_vecs = connect_id_df(result_IDs, doc_vecs)

Compute cosine similarity between retrieved documents and query

In [0]:
def cos_similarity(a, b):
  dot = np.dot(a, b)
  norma = np.linalg.norm(a)
  normb = np.linalg.norm(b)
  cos = dot / (norma * normb)
  return(cos)

In [0]:
cos_sim = []
for i in range(len(result_vecs)):
  doc_vec = result_vecs.loc[i,:].drop(['ID'])
  cos_sim.append(cos_similarity(doc_vec, query_vec))
result_meta['rank'] = cos_sim

Sort retrieved docs by cosine similarity which is proxi for relevance.

In [66]:
result_meta.sort_values('rank', axis=0)

,ID,title,summary,link,published,rank
0,19,week impeachment news three minute,congress heard explosive testimony president d...,https://www.bbc.co.uk/news/world-us-canada-505...,22/11/2019,0.276263


Wrap this in function

In [0]:
def rank_results(query, results):
  query_norm = process_query(query)
  query_vec = average_vectors(word2vec, query_norm)
  result_vecs = connect_id_df(results.ID, doc_vecs)
  cos_sim = []
  for i in range(len(result_vecs)):
    doc_vec = result_vecs.loc[i,:].drop(['ID'])
    cos_sim.append(cos_similarity(doc_vec, query_vec))
  results['rank'] = cos_sim
  results = results.sort_values('rank', axis=0)
  return results

In [0]:
final_result = rank_results("Trump", result_meta)

### Date filtering
User might ask for news from specific day or date range.
For simplicity let's assume the user enters date in format day/month/year. 
Now we can define 2 types of date restrictions:
* single day
* date range - from X/X/X - to X/X/X

Another option of date restriction could be saying 'today'.



Restricing to single day

In [87]:
test = "24/11/2019"

#get news published on "test"
results_single = result_meta[result_meta.published==test].reset_index(drop=True)
results_single.head()

,ID,title,summary,link,published,rank


Restricing to "today"

In [70]:
#get today's date
from datetime import date, timedelta

def get_today():
  today = date.today()
  today = today.strftime("%d/%m/%Y")
  return [today]

results_today = result_meta[result_meta.published.isin(get_today())].reset_index(drop=True)
results_today.head()

,ID,title,summary,link,published,rank


In [71]:
get_today()

['24/11/2019']

Restricting to time interval

In [0]:
def daterange(start, end):
    for n in range(int ((end - start).days)+1):
        yield start + timedelta(n)

def format_date(dt):
  dt = dt.split("/")
  dt = date(int(dt[2]), int(dt[1]), int(dt[0]))
  return(dt)

def date_interval(interval):
  interval = interval.split("-")
  start = format_date(interval[0])
  end = format_date(interval[1])
  interval = []
  for dt in daterange(start, end):
      interval.append(dt.strftime("%d/%m/%Y"))
  return interval

In [73]:
date_interval("15/11/2019 - 16/11/2019")

['15/11/2019', '16/11/2019']

In [74]:
s = "16/11/2019"
len(s)

10

We also need to create the format in which the user should tell the search engine about the date restriction.
Let's pass such request as a new argument for the search function. We can either then simply ask for two separate inputs from the user or find a more elegant solution.

In [0]:
def filter_date(dat, df):
  if dat == "today":
    dat = get_today()
  if len(dat) == 10:
    dat = [dat]
  if len(dat) > 11:
    dat = date_interval(dat)
  if len(dat) is 0:
    return(df)

  result = df[df.published.isin(dat)].reset_index(drop=True)
  return result

In [0]:
filter_date("today", )

### Print results to user

In [0]:
def print_results(result_df):
  for i in range(len(result_df)):
    res = result_df.loc[i, :]
    print(res.title)
    print(res.summary)
    if i == len(result_df):
        print(res.link)
    else:
        print("{}\n" .format(res.link))

In [78]:
print_results(final_result)

week impeachment news three minute 
congress heard explosive testimony president donald trump impeachment inquiry 
https://www.bbc.co.uk/news/world-us-canada-50505712



### Put it all together

In [0]:
def search(query, dat=None):
  result = search_googleish(query)
  result = connect_id_df(result, meta)
  result = rank_results(query, result)

  if dat is not None:
    result = filter_date(dat, result)

  print_results(result)

In [107]:
query = input("Search for:")
dat = input("Date:")
search(query, dat)

Search for:data china leak
Date:
data leak reveals china brainwashes uighur prison camp 
leak document show new evidence china systematic brainwashing uighur detainee 
https://www.bbc.co.uk/news/world-asia-china-50511063



### Automated crawling every hour
Now we want the crawler to scrape new articles every hour by itself. Probably the simplest and most pythonic solution is a while loop and time.sleep().

First we create a list of feeds that we want to scrape.

In [0]:
URLS = ['http://feeds.bbci.co.uk/news/world/rss.xml', 
        'http://feeds.bbci.co.uk/news/uk/rss.xml', 
        'http://www.independent.co.uk/news/uk/rss', 
        'feed:https://rss.nytimes.com/services/xml/rss/nyt/World.xml', 
        'feed:https://rss.nytimes.com/services/xml/rss/nyt/US.xml', 
        'feed://feeds.washingtonpost.com/rss/world?tid=lk_inline_manual_13']

Now we iterate over these urls, scrape data and update the index. NOTE that this is a pseudo-code, the function definitions are stored in their respective scripts.

In [82]:
for url in URLS:
  print ("Crawling {}" .format(url))
  #perc, added = crawl(URL=url, PATH=OUTPUT_DIR)
  print("XXX % of entries were already scraped.\n")
  #update_index_vecs(df=added, index_path=INDEX_PATH, vec_path=VECTOR_PATH)

Crawling http://feeds.bbci.co.uk/news/world/rss.xml
XXX % of entries were already scraped.

Crawling http://feeds.bbci.co.uk/news/uk/rss.xml
XXX % of entries were already scraped.

Crawling http://www.independent.co.uk/news/uk/rss
XXX % of entries were already scraped.

Crawling feed:https://rss.nytimes.com/services/xml/rss/nyt/World.xml
XXX % of entries were already scraped.

Crawling feed:https://rss.nytimes.com/services/xml/rss/nyt/US.xml
XXX % of entries were already scraped.

Crawling feed://feeds.washingtonpost.com/rss/world?tid=lk_inline_manual_13
XXX % of entries were already scraped.



And we put this loop in a while loop. After the for loop is finished we pause the code execution for an hour. Then the code starts from the top again. 
Such while loop would run forever (literally, don't try!!!) so I'm leaving it commented out here.

In [0]:
import time

#while True:
#  for url in URLS:
#   print ("Crawling {}" .format(url))
    #perc, added = crawl(URL=url, PATH=OUTPUT_DIR)
#   print("XXX % of entries were already scraped.\n")
    #update_index_vecs(df=added, index_path=INDEX_PATH, vec_path=VECTOR_PATH)

  #pause for an hour
#  time.sleep(3600)

In real setting you'll need to find some restriction for this while loop. For example let it run for a day can be done as follows (also not recommended to run)

In [109]:
hours = 1

while hours < 24:
#  for url in URLS:
#   print ("Crawling {}" .format(url))
  # perc, added = crawl(URL=url, PATH=OUTPUT_DIR)
#   print("XXX % of entries were already scraped.\n")
  # update_index_vecs(df=added, index_path=INDEX_PATH, vec_path=VECTOR_PATH)

  #pause for 2 seconds - normally this would be 3600 for waiting an hour
   print(hours)
   time.sleep(2)
   hours +=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
